In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from sklearn.metrics import classification_report
from dotenv import load_dotenv
from nltk.probability import FreqDist
from settings import API_KEY

import json



True

In [ ]:
try:

    #llm  = ChatOllama(model="gemma3", temperature=0, base_url="http://192.168.133.192:11435")
    llm = ChatOpenAI(model='gpt-4.1-mini', openai_api_key=API_KEY, temperature=0)
except Exception as e:

    print(f"Erro ao inicializar o Ollama. Certifique-se de que o Ollama está em execução e o modelo 'gemma3' está disponível.")

    print(f"Detalhes do erro: {e}")

    exit()

Erro ao inicializar o Ollama. Certifique-se de que o Ollama está em execução e o modelo 'gemma3' está disponível.
Detalhes do erro: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable


: 

In [3]:


system_prompt_text = """

Você é um especialista em Reconhecimento de Entidades Nomeadas (NER) treinado para identificar e classificar entidades em textos jurídicos brasileiros.
Sua tarefa é analisar o texto fornecido e retornar cada palavra (token) acompanhada de uma etiqueta NER no formato BIO.
As etiquetas possíveis são:

- O: Para palavras que não pertencem a nenhuma entidade.
- B-Organizacao: Para a primeira palavra em uma frase de uma entidade do tipo "Organização" (ex: Ministério Público, Tribunal de Justiça).
- I-Organizacao: Para palavras subsequentes dentro de uma frase para entidade do tipo "Organização".
- B-Jurisprudencia: Para a primeira palavra em uma frase de uma entidade do tipo "Jurisprudência" (ex: Súmula 385 STJ, Recurso Especial nº 1.234.567).
- I-Jurisprudencia: Para palavras subsequentes dentro de uma frse para entidade do tipo "Jurisprudência".
- B-Pessoa: Para a primeira palavra em uma frase de uma entidade do tipo "Pessoa" (ex: João da Silva, Maria Oliveira).
- I-Pessoa: Para palavras subsequentes dentro de uma frase para entidade do tipo "Pessoa".
- B-Tempo: Para a primeira palavra em uma frase de uma entidade do tipo "Tempo" (ex: 01 de janeiro de 2023, segunda-feira, prazo de 15 dias).
- I-Tempo: Para palavras subsequentes dentro de uma frase para entidade do tipo "Tempo".
- B-Local: Para a primeira palavra em uma frase de uma entidade do tipo "Local" (ex: São Paulo, Comarca de Campinas, Brasil).
- I-Local: Para palavras subsequentes dentro de uma frase para entidade do tipo "Local".
- B-Legislacao: Para a primeira palavra em uma frase de uma entidade do tipo "Legislação" (ex: Código Civil, Lei nº 8.666/93, Art. 5º da Constituição Federal).
- I-Legislacao: Para palavras subsequentes dentro de uma frase para entidade do tipo "Legislação".

A entrada será uma lista de listas de string no seguinte formato, exceto pelo termo 'FraseN' no início de cada lista de frases:
[
    Frase1: ['O', 'Dia', 'Está', 'Lindo', 'Hoje'],
    Frase2: ['O', 'Relator', 'Está', 'Ciente', 'Do', 'Contrato'],
    Frase3: ['No', 'caso', 'em', 'debate', 'nestes', 'autos', ',', 'a', 'guarda']
    Frase4: ['Eu', 'sou', 'uma', 'pessoa', 'muito', 'inteligente', '-', 'disse', 'o', 'homem']
    Frase5: ['O', 'arts','.','58','e','67', ',', 'Lei', '8666/94', ',' , '186', 'e', '927', 'do', 'Código', 'Civil', ')', '.' ]
    Frase6: ['Encontra-se', 'em', 'consonância', 'com', 'o', 'fundamento', 'acolhido', 'pelo', 'STF']
    Frase7: ['Confiram-se', 'as', 'seguintes', 'decisões', ':', 'Rcl', '13941', 'MC', '/', ',' , 'Relator', 'Ministros', 'Cezar', 'Peluso', 'DJE', '31/08/2012']
]

A saída DEVE ser estritamente no formato:
{{

    "Frase1": ["O", "O", "O", "O", "B-TEMPO"]
    "Frase2": ["O", "B-PESSOA", "O", "O", "O"]
    "Frase3": ["O","O","O","O","O","O","O","O","O"]
    "Frase4": ["B-PESSOA", "O", "O", "I-PESSOA", "O", "O", "O", "O", "O", "I-PESSOA"]
    "Frase5": ["O", "B-LEGISLACAO", "I-LEGISLACAO", "I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","O","I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","O","O"]
    "Frase6": ["O", "O", "O", "O", "O", "O", "O", "O", "B-ORGANIZAÇÃO"]
    "Frase7": ["O", "O", "O", "O", "O", "B-JURISPRUDENCIA", "I-JURISPRUDENCIA", "I-JURISPRUDENCIA", "I-JURISPRUDENCIA", "I-JURISPRUDENCIA", "O", "O", "O", "B-PESSOA", "I-PESSOA", "O", "B-TEMPO"]
}}

Cada palavra do texto original deve estar em uma nova linha, seguida por um espaço e sua respectiva etiqueta NER.
Se uma palavra for um sinal de pontuação, como vírgula, ponto final ou hífen, atribua a etiqueta "O", a menos que seja parte intrínseca de uma entidade nomeada (o que é raro).
Analise cuidadosamente o contexto para determinar as fronteiras corretas das entidades.
Preste atenção à capitalização, pois pode ser um indicador de entidade, mas não confie apenas nisso.
Entidades podem ser compostas por múltiplas palavras, então note que podemos ter vírgulas ou pontos significando coisas diferentes dependendo do contexto na frase.

NÃO RESPONDA EM MARKDOWN, APENAS EM STRING SIMPLES.
"""

user_prompt_text = """Analise o seguinte texto e forneça a classificação NER para cada palavra no formato especificado:
{texto_entrada}"""

prompt_template = ChatPromptTemplate.from_messages([

    ("system", system_prompt_text),

    ("user", user_prompt_text)

])

chain = prompt_template | llm | StrOutputParser()
texto_exemplo = """
[
    Frase1: ['Número', 'do', 'Acórdão', 'ACÓRDÃO', '1160/2016', '-', 'PLENÁRIO', 'Relator', 'AUGUSTO', 'NARDES', 'Processo', '006.010/2000-4', 'Tipo', 'de', 'processo', 'TOMADA', 'DE', 'CONTAS', 'SIMPLIFICADA', '(', 'TCSP', ')', 'Data', 'da', 'sessão', '11/05/2016', 'Número', 'da', 'ata', '16/2016', 'Relator', 'da', 'deliberação', 'recorrida', 'Ministra', 'Ana', 'Arraes', '.']
]
"""



In [4]:
def load_conll_data(filepath):
    """
    Loads data from a file in CoNLL format.
    """
    sentences, tag_sequences = [], []
    current_words, current_tags = [], []
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line:
                if len(line.split()) == 2:
                    word, tag = line.split()
                    current_words.append(word)
                    current_tags.append(tag)
            else:
                if current_words:
                    sentences.append(current_words)
                    tag_sequences.append(current_tags)
                    current_words, current_tags = [], []
    if current_words:
        sentences.append(current_words)
        tag_sequences.append(current_tags)
    return sentences, tag_sequences


In [5]:
test_sentences, test_tags = load_conll_data("G:\\Meu Drive\\Projects-IA\\NER\\archive\\lener-br\\model\\data\\test.txt")

In [6]:
print(f"Número de frases de treino: {len(test_sentences)}")
print("Exemplo de frase de treino:")
print(len(test_sentences[34]))
print("Etiquetas correspondentes:")
print(len(test_tags[34]))


Número de frases de treino: 1389
Exemplo de frase de treino:
26
Etiquetas correspondentes:
26


In [7]:
def format_sentences_for_llm(sentences_batch):
    """
    Formats a batch of sentences into the string format required by the LLM prompt.
    """
    formatted_list = []
    for i, sentence in enumerate(sentences_batch, 1):
        # Escape single quotes and backslashes for proper string representation
        cleaned_sentence = [word.replace("\\", "\\\\").replace("'", "\\'") for word in sentence]
        formatted_list.append(f"    Frase{i}: {cleaned_sentence}")
    return "[\n" + ",\n".join(formatted_list) + "\n]"

In [8]:
# Initialize lists to store all true and predicted tags
all_true_tags = []
all_predicted_tags = []

# Process sentences in batches of 10
batch_size = 10
for i in range(0, len(test_sentences), batch_size):
    # Get the current batch of sentences and corresponding true tags
    sentence_batch = test_sentences[i:i + batch_size]
    true_tags_batch = test_tags[i:i + batch_size]

    print(f"\nProcessing batch {i//batch_size + 1}/{(len(test_sentences) + batch_size - 1)//batch_size}...")

    # Format the batch for the LLM prompt
    formatted_input = format_sentences_for_llm(sentence_batch)

    try:
        # Invoke the LLM chain
        response = chain.invoke({"texto_entrada": formatted_input})

        # Clean and parse the JSON response
        parsed_response = json.loads(response)

        if parsed_response:
            # Iterate through the original batch to ensure correct order
            for j, (original_sentence, true_tag_list) in enumerate(zip(sentence_batch, true_tags_batch)):
                frase_key = f"Frase{j+1}"
                predicted_tag_list = parsed_response.get(frase_key)

                # Validate the prediction
                if predicted_tag_list and len(predicted_tag_list) == len(original_sentence):
                    all_predicted_tags.extend(predicted_tag_list)
                    all_true_tags.extend(true_tag_list)
                else:
                    # Handle cases where a prediction is missing or has the wrong length
                    print(f"  - Warning: Mismatch or missing prediction for sentence {i+j+1}. Skipping.")
                    # To keep alignment, you could add 'O' tags, but skipping is safer
                    # For example: all_predicted_tags.extend(['O'] * len(true_tag_list))

    except Exception as e:
        print(f"--- An error occurred during processing batch starting at sentence {i+1} ---")
        print(f"Error details: {e}")

# --- Generate the Classification Report ---



Processing batch 1/139...
  - Warning: Mismatch or missing prediction for sentence 1. Skipping.
  - Warning: Mismatch or missing prediction for sentence 2. Skipping.
  - Warning: Mismatch or missing prediction for sentence 5. Skipping.
  - Warning: Mismatch or missing prediction for sentence 8. Skipping.

Processing batch 2/139...
  - Warning: Mismatch or missing prediction for sentence 11. Skipping.
  - Warning: Mismatch or missing prediction for sentence 13. Skipping.
  - Warning: Mismatch or missing prediction for sentence 14. Skipping.
  - Warning: Mismatch or missing prediction for sentence 15. Skipping.
  - Warning: Mismatch or missing prediction for sentence 16. Skipping.

Processing batch 3/139...
--- An error occurred during processing batch starting at sentence 21 ---
Error details: Expecting value: line 1 column 1 (char 0)

Processing batch 4/139...
--- An error occurred during processing batch starting at sentence 31 ---
Error details: Expecting value: line 1 column 1 (cha

In [9]:
all_tags = [tag for tags_seq in test_tags for tag in tags_seq]
tag_counts = FreqDist(all_tags)
vocab_tags = list(tag_counts.keys())
vocab_tags

['O',
 'B-JURISPRUDENCIA',
 'I-JURISPRUDENCIA',
 'B-ORGANIZACAO',
 'B-PESSOA',
 'I-PESSOA',
 'B-TEMPO',
 'I-ORGANIZACAO',
 'B-LOCAL',
 'I-LOCAL',
 'B-LEGISLACAO',
 'I-LEGISLACAO',
 'I-TEMPO']

In [11]:
print("\n--- Final Classification Report ---")

# Flatten the lists of lists into a single list for the report


# Get all unique labels from both true and predicted tags to ensure they are all included in the report
labels = ['B-JURISPRUDENCIA', 'B-LEGISLACAO', 'B-LOCAL', 'B-ORGANIZACAO', 'B-PESSOA', 'B-TEMPO', 'I-JURISPRUDENCIA', 'I-LEGISLACAO', 'I-LOCAL', 'I-ORGANIZACAO', 'I-PESSOA', 'I-TEMPO', 'O']

# Generate and print the report
if all_true_tags and all_predicted_tags:
    report = classification_report(
        all_true_tags,
        all_predicted_tags,
        labels=vocab_tags,
        digits=2,
        zero_division=0
    )
    print(report)
else:
    print("Could not generate report: No valid predictions were collected.")


--- Final Classification Report ---
                  precision    recall  f1-score   support

               O       0.97      0.78      0.87      3240
B-JURISPRUDENCIA       0.15      0.55      0.23        22
I-JURISPRUDENCIA       0.09      0.30      0.14        47
   B-ORGANIZACAO       0.23      0.27      0.25        33
        B-PESSOA       0.44      0.37      0.40        19
        I-PESSOA       0.68      0.52      0.59        54
         B-TEMPO       0.16      0.62      0.25        13
   I-ORGANIZACAO       0.50      0.15      0.24        71
         B-LOCAL       0.44      1.00      0.62         4
         I-LOCAL       0.04      1.00      0.07         1
    B-LEGISLACAO       0.36      0.75      0.48        28
    I-LEGISLACAO       0.67      0.46      0.55       156
         I-TEMPO       1.00      0.72      0.84        18

       micro avg       0.84      0.74      0.79      3706
       macro avg       0.44      0.58      0.42      3706
    weighted avg       0.91      